In [18]:
using CSV, JuMP, Gurobi, DataFrames

In [19]:
energy_matrix = CSV.read("HW2_data/energy.csv", DataFrame) |> df -> select(df, Not(1)) |> Matrix
capacity_vector = CSV.read("HW2_data/capacity.csv", DataFrame) |> df -> select(df, Not(1)) |> Matrix

20×1 Matrix{Float64}:
  23.0
  24.0
  30.0
   4.0
  70.0
  87.0
  75.0
 107.0
  43.0
  52.0
   4.0
  20.0
  49.0
  69.0
 141.0
  32.0
  35.0
   4.0
  23.0
 108.0

In [20]:
energy_matrix

1000×20 Matrix{Float64}:
 58.4516  56.4218  53.3182  61.1425  …  58.5737  47.434   60.5322  53.7255
 47.1307  49.9741  49.4615  53.4576     45.5567  44.2967  43.7656  51.6316
 72.7165  64.8131  62.9752  76.3534     59.1579  56.775   61.0008  60.1781
 44.5003  51.6669  45.6587  44.7678     55.7139  41.9219  39.5938  45.9446
 48.9771  41.3224  54.4063  53.2036     57.0393  53.6506  41.1886  56.2313
 14.4386  19.9577  18.9604  21.8064  …  18.9657  18.891   20.1127  18.9616
 72.9008  66.1734  71.4013  64.5653     67.8859  62.7328  65.5753  59.0646
 31.1955  29.6268  25.2512  26.9403     32.0769  27.4883  28.1615  26.8298
 73.8362  79.5029  72.8443  83.7081     78.4482  65.0861  78.7875  77.2289
 74.7685  82.2525  71.9211  75.4779     70.681   75.8865  76.0492  71.2974
 48.544   34.5686  42.3093  39.9429  …  38.6825  40.2174  42.9395  45.8979
 32.0782  31.5283  33.1421  33.7846     34.6609  38.5242  37.7509  43.3149
 16.0317  15.4004  18.9355  17.5292     16.7998  17.6602  18.5714  16.671
 

# Part b

In [21]:
num_jobs, num_machines = size(energy_matrix)

# Initialize the model
model = Model(Gurobi.Optimizer)

# Decision variables: x[i, j] is the fraction of job i assigned to machine j
@variable(model, 0 <= x[1:num_jobs, 1:num_machines] <= 1)

# Objective: Minimize total energy consumption
@objective(model, Min, sum(energy_matrix[i, j] * x[i, j] for i in 1:num_jobs, j in 1:num_machines))

# Constraints:
# 1. Each job must be fully assigned (sum of fractions across machines must be 1)
@constraint(model, [i=1:num_jobs], sum(x[i, j] for j in 1:num_machines) == 1)

# 2. Machine capacity constraints
@constraint(model, [j=1:num_machines], sum(x[i, j] for i in 1:num_jobs) <= capacity_vector[j])

# Solve the problem
optimize!(model)

# (i) Optimal energy consumption
optimal_energy_consumption = objective_value(model)
println("Optimal energy consumption: ", optimal_energy_consumption)

# (ii) Energy consumption if each job is assigned to the machine with the lowest energy consumption
min_energy_consumption_per_job = sum(minimum(energy_matrix[i, :]) for i in 1:num_jobs)
println("Energy consumption without machine capacity constraints: ", min_energy_consumption_per_job)

# (iii) Number of jobs not fully assigned to the machine with the lowest energy consumption
x_optimal = value.(x)
jobs_not_assigned_to_best_machine = sum([argmin(energy_matrix[i, :]) != argmax(x_optimal[i, :]) for i in 1:num_jobs])
println("Number of jobs not assigned to the machine with the lowest energy consumption: ", jobs_not_assigned_to_best_machine)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1020 rows, 20000 columns and 40000 nonzeros
Model fingerprint: 0x5ad88959
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 1020 rows, 20000 columns, 40000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+04
 Factor NZ  : 2.168e+04 (roughly 9 MB of memory)
 Factor Ops : 4.729e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual    

In [22]:
x_check = value.(x)

1000×20 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [23]:
# (iv) Output for each machine a sorted list of the jobs assigned to it
println("\nMachine-to-Job Assignments:")
for j in 1:num_machines
    # Collect the jobs assigned to machine j (where x[i, j] == 1)
    jobs_assigned_to_machine = [i for i in 1:num_jobs if x_optimal[i, j] == 1.0]
    
    # Sort the list of jobs
    sorted_jobs = sort(jobs_assigned_to_machine)
    
    # Print the sorted list of jobs for machine j
    println("Machine $j has the following jobs assigned: $sorted_jobs")
end


Machine-to-Job Assignments:
Machine 1 has the following jobs assigned: [42, 54, 109, 123, 136, 162, 250, 322, 328, 385, 429, 503, 525, 542, 617, 687, 745, 777, 800, 910, 940, 950, 973]
Machine 2 has the following jobs assigned: [5, 47, 52, 110, 120, 207, 293, 335, 377, 492, 499, 570, 581, 655, 672, 679, 798, 812, 819, 832, 840, 907, 914, 962]
Machine 3 has the following jobs assigned: [71, 74, 99, 186, 188, 243, 292, 349, 391, 403, 406, 480, 488, 504, 533, 600, 644, 654, 658, 670, 769, 817, 823, 862, 878, 883, 885, 951, 954, 963]
Machine 4 has the following jobs assigned: [189, 387, 392, 498]
Machine 5 has the following jobs assigned: [21, 68, 102, 104, 107, 115, 177, 185, 227, 229, 245, 249, 262, 266, 280, 281, 284, 286, 301, 304, 316, 338, 369, 383, 412, 431, 434, 438, 448, 451, 453, 469, 470, 476, 506, 519, 558, 574, 577, 642, 652, 674, 675, 676, 678, 681, 698, 702, 710, 720, 721, 751, 761, 773, 774, 775, 785, 787, 793, 831, 843, 871, 876, 879, 895, 896, 911, 912, 933, 986]
Machine

# Part e

In [24]:
energy_matrix = CSV.read("HW2_data/energy.csv", DataFrame) |> df -> select(df, Not(1)) |> Matrix
utilization_matrix = CSV.read("HW2_data/utilization.csv", DataFrame) |> df -> select(df, Not(1)) |> Matrix
max_util_vector = CSV.read("HW2_data/maxutil.csv", DataFrame) |> df -> select(df, Not(1)) |> Matrix

20×1 Matrix{Float64}:
 1170.6530507150087
 1356.7292590989266
  774.6407399067323
  583.42002453427
  806.3621979348665
  327.14308351538904
 3895.1172973703497
  591.3964093287686
 1968.62736382538
  376.4551601091232
 1964.1195955088235
  754.0803116165797
 3820.865117789083
  317.8379429234617
 1939.650469596368
  714.5178536320536
 1460.1424978957966
 1986.400958382396
  290.124209071122
 4947.224624995945

In [25]:
num_jobs, num_machines = size(energy_matrix)

# Initialize the model
model = Model(Gurobi.Optimizer)

# Decision variables: x[i, j] is the fraction of job i assigned to machine j
@variable(model, 0 <= x[1:num_jobs, 1:num_machines] <= 1)

# Objective: Minimize total energy consumption
@objective(model, Min, sum(energy_matrix[i, j] * x[i, j] for i in 1:num_jobs, j in 1:num_machines))

# Constraints:
# 1. Each job must be fully assigned (sum of fractions across machines must be 1)
@constraint(model, [i=1:num_jobs], sum(x[i, j] for j in 1:num_machines) == 1)

# 2. Machine capacity constraints
@constraint(model, machine_utilization[j=1:num_machines], sum(utilization_matrix[i, j] * x[i, j] for i in 1:num_jobs) <= max_util_vector[j])

# Solve the problem
optimize!(model)

# (i) Optimal energy consumption
optimal_energy_consumption = objective_value(model)
println("Optimal energy consumption: ", optimal_energy_consumption)

# (ii) Energy consumption if each job is assigned to the machine with the lowest energy consumption
min_energy_consumption_per_job = sum(minimum(energy_matrix[i, :]) for i in 1:num_jobs)
println("Energy consumption without machine capacity constraints: ", min_energy_consumption_per_job)

# (iii) Number of jobs not fully assigned to the machine with the lowest energy consumption
x_optimal = value.(x)
jobs_not_assigned_to_best_machine = sum([maximum(x_optimal[i, :]) != 1 || argmin(energy_matrix[i, :]) != argmax(x_optimal[i, :]) for i in 1:num_jobs])
println("Number of jobs not assigned to the machine with the lowest energy consumption: ", jobs_not_assigned_to_best_machine)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1020 rows, 20000 columns and 40000 nonzeros
Model fingerprint: 0xbaaba343
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [7e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Presolve time: 0.01s
Presolved: 1020 rows, 20000 columns, 40000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+04
 Factor NZ  : 2.168e+04 (roughly 9 MB of memory)
 Factor Ops : 4.729e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual    

In [33]:
# (iv) Output for each machine a sorted list of the jobs assigned to it
println("\nMachine-to-Job Assignments:")
for j in 1:num_machines
    # Collect the jobs assigned to machine j (where x[i, j] == 1)
    jobs_assigned_to_machine = [i for i in 1:num_jobs if x_optimal[i, j] == 1.0]
    
    # Sort the list of jobs
    sorted_jobs = sort(jobs_assigned_to_machine)
    
    # Print the sorted list of jobs for machine j
    println("Machine $j has the following jobs assigned: $sorted_jobs")
end


Machine-to-Job Assignments:
Machine 1 has the following jobs assigned: [43, 70, 74, 91, 120, 134, 210, 216, 242, 284, 319, 326, 369, 391, 396, 406, 445, 450, 458, 464, 488, 494, 531, 546, 572, 593, 595, 599, 640, 642, 647, 651, 664, 668, 676, 722, 732, 758, 759, 790, 793, 797, 801, 850, 853, 876, 877, 938]
Machine 2 has the following jobs assigned: [53, 63, 108, 109, 116, 122, 141, 151, 156, 161, 183, 193, 219, 226, 250, 279, 293, 321, 360, 361, 398, 401, 407, 438, 492, 514, 516, 539, 548, 576, 581, 597, 621, 641, 653, 656, 719, 725, 760, 783, 812, 836, 858, 874, 919, 927, 944, 962, 973, 974, 980]
Machine 3 has the following jobs assigned: [14, 23, 62, 123, 159, 164, 178, 185, 288, 300, 328, 330, 399, 425, 431, 465, 502, 515, 536, 538, 554, 658, 726, 750, 787, 846, 848, 879, 885, 886, 887, 987]
Machine 4 has the following jobs assigned: [34, 228, 237, 243, 324, 342, 402, 418, 459, 499, 556, 561, 566, 583, 639, 723, 766, 803, 816, 817, 849, 889, 975]
Machine 5 has the following jobs as

# Part f

In [35]:
# Get the values of the dual variables for the machine utilization constraints from the model in Part e

dual_values = dual.(machine_utilization)

20-element Vector{Float64}:
 -0.6877375258524322
 -0.6276405251108662
 -0.7475845837313916
 -0.9423350572679897
 -0.7983120635098678
 -0.9559487912487902
 -0.47639998175706827
 -0.8631283570921323
 -0.5626604689530459
 -0.8619179713277901
 -0.6017256987589229
 -0.8353484646469141
 -0.4694121774821506
 -1.017551184215603
 -0.5573576034764329
 -0.7586473115817243
 -0.6567023032387405
 -0.6083138421319758
 -1.0561672403707179
 -0.4128887105714022

In [36]:
df = DataFrame(Dual_Values = dual_values)
CSV.write("dual_values.csv", df)

"dual_values.csv"